<a href="https://colab.research.google.com/github/SifaMansuri/CE888/blob/main/TweetEval_final_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import transformers
import json

from tqdm import tqdm
from pathlib import Path 
from torch.utils.data import DataLoader,Dataset
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
emoji_data = "/content/drive/MyDrive/dataset/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/datasets/emoji/"
emotion_data = "/content/drive/MyDrive/dataset/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/datasets/emotion"
hate_data = "/content/drive/MyDrive/dataset/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/tweeteval-8df207ad6fcdab017000ad5196f797cd21f9a30f/datasets/hate"

In [ ]:
def create_data(path,train_data = True,val_data = False,test_data = False):
    if train_data:
        file = open("{}/train_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()

        file = open("{}/train_labels.txt".format(path),"r")
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()
        
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    if test_data:
        file = open("{}/test_labels.txt".format(path),"r")
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()

        file = open("{}/test_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()
        
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    if val_data:
        file = open("{}/val_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()

        file = open("{}/val_labels.txt".format(path),'r')
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    

In [ ]:
emoji_files = os.listdir(emoji_data)
emotion_files = os.listdir(emotion_data)
hate_files = os.listdir(hate_data)

In [ ]:
train_emoji = create_data(path=emoji_data)
val_emoji = create_data(path = emoji_data,val_data=True,train_data = False)
test_emoji = create_data(path=emoji_data,train_data=False,val_data=False,test_data=True)

train_emotion = create_data(path=emotion_data)
val_emotion = create_data(path = emotion_data,val_data=True,train_data = False)
test_emotion = create_data(path=emotion_data,train_data=False,val_data=False,test_data=True)

train_hate = create_data(path=hate_data)
val_hate = create_data(path = hate_data,val_data=True,train_data = False)
test_hate = create_data(path=hate_data,train_data=False,val_data=False,test_data=True)

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-05
VAL_BATCH_SIZE = 4
TEST_BATCH_SIZE = 8

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base',truncation = True,do_lower_case = True)

In [ ]:
class Data(Dataset):
    def __init__(self,dataframe,tokenize):
        self.data = dataframe
        self.tokenize = tokenize
        self.phrase = self.data.Phrases
        self.target = self.data.Label
        
    def __len__(self):
        return len(self.phrase)
        
    def __getitem__(self,index):
        text = str(self.phrase[index])
        text = " ".join(text.split())
        
        inputs = self.tokenize.encode_plus(text,None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_token_type_ids=True)
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.target[index], dtype=torch.float)
        }

In [ ]:
train_data_emoji = Data(train_emoji,tokenizer)
val_data_emoji = Data(val_emoji,tokenizer)
test_data_emoji = Data(test_emoji,tokenizer)

train_data_emotion = Data(train_emotion,tokenizer)
val_data_emotion = Data(val_emotion,tokenizer)
test_data_emotion = Data(test_emotion,tokenizer)

train_data_hate = Data(train_hate,tokenizer)
val_data_hate = Data(val_hate,tokenizer)
test_data_hate = Data(test_hate,tokenizer)

In [ ]:
# Recemmonded To train the data the batch size = 1 , shuffle = True
# and for validation and test put batch size = 1 and shuffle = False, because later on it will be difficult to predict the result
# I have trained with shuffle = True.
# And I can't predict the F1 Score for the test data. So, Recemmonded that to do shuffle = False while working with Val and test data
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
val_params = {'batch_size': VAL_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

In [ ]:
training_emoji_loader = DataLoader(train_data_emoji, **train_params)
val_emoji_loader = DataLoader(val_data_emoji,**val_params)
test_emoji_loader = DataLoader(test_data_emoji,**test_params)

training_emotion_loader = DataLoader(train_data_emotion, **train_params)
val_emotion_loader = DataLoader(val_data_emotion,**val_params)
test_emotion_loader = DataLoader(test_data_emotion,**test_params)

training_hate_loader = DataLoader(train_data_hate, **train_params)
val_hate_loader = DataLoader(val_data_hate,**val_params)
test_hate_loader = DataLoader(test_data_hate,**test_params)

In [ ]:
class RoBERTa(torch.nn.Module):
    def __init__(self,no_labels):
        super(RoBERTa,self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base')
        self.pre_classifier = torch.nn.Linear(768,768)
        self.Dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768,no_labels)
        
    def forward(self,input_ids,attention_mask,token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.Dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
emoji_model = RoBERTa(no_labels=20)
emotion_model = RoBERTa(no_labels=4)
hate_model = RoBERTa(no_labels=2)

In [ ]:
emoji_model.to(device)
emotion_model.to(device)
hate_model.to(device)

RoBERTa(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
def optimizer_(model):
  return torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(model,epoch,training_loader):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer = optimizer_(model)
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer = optimizer_(model)
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(emoji_model,epoch,training_emoji_loader)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 2.9201629161834717
Training Accuracy per 5000 steps: 0.0


1252it [1:29:21,  3.97s/it]

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(emotion_model,epoch,training_emotion_loader)

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(hate_model,epoch,training_hate_loader)

In [ ]:
output_model_file = 'TweetEval-emoji.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)



output_model_file = 'TweetEval-emotion.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)



output_model_file = 'TweetEval-hate.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [ ]:
acc = valid(emoji_model, val_emoji_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(emotion_model, val_emotion_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(hate_model, val_hate_loader)
print("Accuracy on val data = %0.2f%%" % acc)

In [ ]:
acc = valid(emoji_model, test_emoji_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(emotion_model, test_emotion_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(hate_model, test_hate_loader)
print("Accuracy on val data = %0.2f%%" % acc)

In [ ]:
def predict(model, test_loader):
    model.eval()
    output = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(test_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            output.append(outputs)
    return output

In [ ]:
emoji_pred = predict(emoji_model,test_emoji_loader)
emotion_pred = predict(emotion_model,test_emotion_loader)
hate_pred = predict(hate_model,test_hate_loader)

In [ ]:
emoji_predictions = list()
for i in range(len(emoji_pred)):
  for j in range(TEST_BATCH_SIZE):
    out_arr = emoji_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    emoji_predictions.append(max_id)

In [ ]:
len(emoji_predictions)

In [ ]:
file = open('emoji_pred.txt','w')
for i in emoji_predictions:
  file.write(str(i))
  print("\n")
file.close()

In [ ]:
emotion_predictions = list()
for i in range(len(emotion_pred)):
  for j in range(len(emotion_pred[i])):
    out_arr = emotion_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    emotion_predictions.append(max_id)

In [ ]:
len(emotion_predictions)

In [ ]:
file = open('emotion_pred.txt','w')
for i in emotion_predictions:
  file.write(str(i))
  print("\n")
file.close()

In [ ]:
hate_predictions = list()
for i in range(len(hate_pred)):
  for j in range(len(hate_pred[i])):
    out_arr = hate_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    hate_predictions.append(max_id)

In [ ]:
len(hate_predictions)

In [ ]:
file = open('hate_pred.txt','w')
for i in hate_predictions:
  file.write(str(i))
  print("\n")
file.close()